In [71]:
import sys
import pandas as pd
import numpy as np
from nltk.book import *
from nltk.corpus import stopwords
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
import random
from nltk import classify
from nltk import NaiveBayesClassifier
from nltk.tokenize import word_tokenize
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim
sns.set_context('notebook')
import warnings
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from nltk.corpus import stopwords
import requests
import json
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [77]:
#### get all csv dataframes for trending
US_trending_df = pd.read_csv('youtube-new/USvideos.csv') #USA
CA_trending_df = pd.read_csv('youtube-new/CAvideos.csv') #CANADA
DE_trending_df = pd.read_csv('youtube-new/DEvideos.csv') #GERMANY
FR_trending_df = pd.read_csv('youtube-new/FRvideos.csv') #FRANCE
GB_trending_df = pd.read_csv('youtube-new/GBvideos.csv') #GREAT BRITAIN
IN_trending_df = pd.read_csv('youtube-new/INvideos.csv') #INDIA

# JP_trending_df = pd.read_csv('youtube-new/JPvideos.csv', encoding="UTF-8") #JAPAN

# KR_trending_df = pd.read_csv('youtube-new/KRvideos.csv') #SOUTH KOREA

# MX_trending_df = pd.read_csv('youtube-new/MXvideos.csv') #MEXICO

# RU_trending_df = pd.read_csv('youtube-new/RUvideos.csv') #RUSSIA

list_of_all_trending_dfs = [US_trending_df, CA_trending_df, DE_trending_df, FR_trending_df, GB_trending_df, IN_trending_df]
full_trending_df = pd.concat(list_of_all_dfs)

In [68]:
f = open("apiKey", "r")
key = f.read()
## for each videoId, find a related video
def do_search_youtube_request(videoId):
    url = "https://www.googleapis.com/youtube/v3/search?part=snippet&maxResults=2&order=relevance&relatedToVideoId={}&type=video&videoDefinition=any&key={}".format(videoId, key)
    r = requests.get(url)
    return r

## given a set of videoIds, find insights (statistics, tags, etc)
def find_video_insights(videoIds):
    print(videoIds)
    url = 'https://www.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id={}&key={}'.format(videoIds, key)
    r = requests.get(url)
    return r

## call this with 1 country at a time 
def process_youtube_requests(videoIds):
    df = pd.DataFrame(columns=['video_id', 'title', 'channel_title', 'category_id', 'publish_time', 'tags', 
                               'views', 'likes', 'dislikes', 'comment_count', 'description'])
    relatedVideoIds = []
    responses = []
    for videoId in videoIds:
        try: 
            response = do_search_youtube_request(videoId)
            if (response.status_code == 200):
                r1 = response.json()
                relatedVideoId = r1['items'][0]['id']['videoId']
                relatedVideoIds.append(str(relatedVideoId))
            else:
                print(response.status_code)
        except:
            print ("Something went wrong here! 2")
    print (relatedVideoIds)
    videoIdsStr = '%2C'.join([str(elem) for elem in relatedVideoIds])
    r2 = find_video_insights(videoIdsStr)
    if (r2.status_code == 200):
        r = r2.json()
        i = 0
        while (i < len(relatedVideoIds)):
            try:
                id = relatedVideoIds[i]
                title = (r['items'][i]['snippet']['title'])
                channel_title = (r['items'][i]['snippet']['channelTitle'])
                category_id = (r['items'][i]['snippet']['categoryId'])
                publish_time = (r['items'][i]['snippet']['publishedAt'])
                tags = '|'.join((r['items'][i]['snippet']['tags']))
                views = (r['items'][i]['statistics']['viewCount'])
                likes = (r['items'][i]['statistics']['likeCount'])
                dislikes = (r['items'][i]['statistics']['dislikeCount'])
                comment_count = (r['items'][i]['statistics']['commentCount'])
                description = (r['items'][i]['snippet']['description'])
                data = {'video_id': id, 'title': title, 'channel_title': channel_title, 'category_id' : category_id,
                       'publish_time' : publish_time, 'tags' : tags, 'views' : views, 'likes' : likes, 'dislikes' : dislikes,
                       'comment_count' : comment_count, 'description' : description}
                df = df.append(data, ignore_index = True)
            except:
                print("Something went wrong! 3")
            i = i + 1
    return df
        

In [ ]:
n = 100
US_trending_videoIds = US_trending_df.sample(n)['video_id'].tolist()
CA_trending_videoIds = CA_trending_df.sample(n)['video_id'].tolist()
DE_trending_videoIds = DE_trending_df.sample(n)['video_id'].tolist()
FR_trending_videoIds = FR_trending_df.sample(n)['video_id'].tolist()
GB_trending_videoIds = GB_trending_df.sample(n)['video_id'].tolist()
IN_trending_videoIds = IN_trending_df.sample(n)['video_id'].tolist()

In [ ]:
## do following requests separately with a new API Key

In [ ]:
not_trending_us_df = process_youtube_requests(US_trending_videoIds)

In [ ]:
not_trending_ca_df = process_youtube_requests(CA_trending_videoIds)

In [ ]:
not_trending_de_df = process_youtube_requests(DE_trending_videoIds)

In [ ]:
not_trending_fr_df = process_youtube_requests(FR_trending_videoIds)

In [ ]:
not_trending_gb_df = process_youtube_requests(GB_trending_videoIds)

In [ ]:
not_trending_in_df = process_youtube_requests(IN_trending_videoIds)

In [ ]:
list_of_all_nontrending_dfs = [non_trending_us_df, not_trending_ca_df, not_trending_de_df, 
                              not_trending_fr_df, not_trending_gn_df, not_trending_in_df]
full_nontrending_df = pd.concat(list_of_all_nontrending_dfs)

In [78]:
def insert_category_field(df, json_file):
    # add a category column using the category id from the json file
    id_to_category = {}
    with open(json_file, 'r') as f:
        data = json.load(f)
        for category in data['items']:
            id_to_category[category['id']] = category['snippet']['title']
        df.insert(4, 'category', us['category_id'].map(id_to_category))

In [ ]:
## insert new category field into dataframes
insert_category_field(US_trending_df, "US_category_id.json")
insert_category_field(CA_trending_df, "CA_category_id.json")
insert_category_field(DE_trending_df, "DE_category_id.json")
insert_category_field(FR_trending_df, "FR_category_id.json")
insert_category_field(GB_trending_df, "GB_category_id.json")
insert_category_field(IN_trending_df, "IN_category_id.json")

insert_category_field(not_trending_us_df, "US_category_id.json")
insert_category_field(not_trending_ca_df, "CA_category_id.json")
insert_category_field(not_trending_de_df, "DE_category_id.json")
insert_category_field(not_trending_fr_df, "FR_category_id.json")
insert_category_field(not_trending_gb_df, "GB_category_id.json")
insert_category_field(not_trending_in_df, "IN_category_id.json")

In [ ]:
## do following for both trending and nontrending datasets and/or per country
## do good visualizations for all!!!

## find likes to dislikes ratio ---> shows where people have the greatest divide and where people agree the most 
## which categories are the most popular? do highest average amongst likes and views ----> shows which type of videos people enjoy the most
## most trending videos in each country? do average amongst likes, views ------> shows most trending videos per country
## how long did a video stay trending? how long did it take to become trending?
## which country has the most active participation and online presence across trending and nontrending? ----> min, max, std, quartiles, counts, etc
## correlation between views, likes, dislikes, and comments
## get most frequent tags ----> run sentiment analysis on tags, run LDA model on tags
